# *Procedure* for building " *The df_ib DataFrame from ib770 & ib970 combined with meta_data*

# Import Subroutines and Settings

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [3]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


# Startup only Create  **df_ib_fls_tmplt**  ie  *[COL_NMS = meta+ib770dat + ib970dat]*

## Add the columns of df_77 & df_97 then delete Column names and add meta cols and Store as data frames in pickle called *df_ib_tst_nms*

In [4]:
# this def function STRIPS COL NMBRS  column #s from col heads used only in making the files

def strip_col_numbers(col_names):
    """
    Remove leading numbers and dots from column names.
    Example: '244. 50kHz-Whole Body Phase Angle_Z score' 
             -> '50kHz-Whole Body Phase Angle_Z score'
    """
    return [re.sub(r'^\d+\.\s*', '', name) for name in col_names]

### ib77  This segment Reads the data from the excel file and computes names of the column head, strips them of numbers and Puts them in a data frame and records them in a pickle. *"df_ib77_raw_nms"*

In [5]:
# loads the new from the 770
ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# strip numbers off So that the identical names in IB77 are the same as IB97
ib77_raw_nms = strip_col_numbers(ib77_raw)
# Now we'll make it into a data frame so that we will print it in a column which obey list [numbers separated by commas]
df_ib77_raw_nms = pd.DataFrame(ib77_raw_nms)

# Load to pickle file in the working directory
df_ib77_raw_nms.to_pickle("df_ib77_raw_nms.pkl")

# For verification the picket file and print its value.
df_ib77_raw_nms = pd.read_pickle("df_ib77_raw_nms.pkl")

# verify 
# print(ib77_raw_nms)
# verify print("df_ib77_raw_nms \n",df_ib77_raw_nms)

✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725_20251217075000.csv with utf-8
✅ Loaded 091725_20251218081604.csv with utf-8
✅ Loaded 091725_20251216233809.csv with utf-8
✅ Loaded 091725_20251215080038.csv with utf-8
✅ Loaded 091725-1_20251212221713.csv with utf-8
✅ Loaded 091725_20251219080729.csv with utf-8
✅ Loaded 091725_20251214090336.csv with utf-8
✅ Loaded 091725_20251218212747.csv with utf-8
✅ Loaded 091725_20251219232758.csv with utf-8
✅ Loaded 091725_20251215233324.csv with utf-8
✅ Loaded 091725_20251216064808.csv with utf-8
✅ Loaded 091725-1_20251211092610.csv with utf-8
✅ Loaded 091725_20251217230745.csv with utf-8
✅ Loaded 091725_20251220085024.csv with utf-8
✅ Loaded 091725-1_20251212084231.csv with utf-8
✅ [imported_dataframe] Final DataFrame: 17 rows from 17 files.
💾 Saved to pickle: /home/bhuns/JL_2/imported_dataframe.pkl


### ib97 This segment Reads the data from the excel file and computes names of the column head, strips them of numbers and Puts them in a data frame and records them in a pickle. *"df_ib97_raw_nms"*

In [6]:
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="*"
)
# strip numbers off So that the identical names in IB97 are the same as IB97
ib97_raw_nms = strip_col_numbers(ib97_raw)
# Now we'll make it into a data frame so that we will print it in a column which obey list [numbers separated by commas]
df_ib97_raw_nms = pd.DataFrame(ib97_raw_nms)

# Load to pickle file in the working directory
df_ib97_raw_nms.to_pickle("df_ib97_raw_nms.pkl")

# For verification the picket file and print its value.
df_ib97_raw_nms = pd.read_pickle("df_ib97_raw_nms.pkl")

# verify 
# print(ib97_raw_nms)
# verify 
# print("df_ib97_raw_nms \n",df_ib97_raw_nms)


✅ Loaded 091725_bdailyn_20250922225826.csv with ISO-8859-1
✅ [imported_dataframe] Final DataFrame: 1 rows from 1 files.
💾 Saved to pickle: /home/bhuns/JL_2/imported_dataframe.pkl


### Both the 77 and the 90 test datasets are convrted to dfs In order to determine *df_ib_tst_nms* Which is a single column data frame that all of the column names of the *df_ib_tst*

#### ✅ Concatenate → Preserve Order → Drop Duplicates

In [7]:
####  Concatenate → Preserve Order → Drop Duplicates mmmmmmmmmnnnnnnnllllll
# Concatenate in the required order
df_ib_tst_nms = pd.concat(
    [df_ib77_raw_nms, df_ib97_raw_nms],
    axis=0,
    ignore_index=True
)
# print(df_ib_tst_nms)
# Remove duplicate rows, keeping the first occurrence (from df_ib77_raw_nms)
df_ib_tst_nms = df_ib_tst_nms.drop_duplicates(keep="first")
# verify 
# print(df_ib_tst_nms)

In [8]:
# Verifies that the duplicates have been removed and that the list has been compacted and there are not duplicates of 770 and 970 Col
df_ib_tst_nms[df_ib_tst_nms.duplicated()]

,0


### *df_ib_tst_nms* must be converted to a simple list in order to made equal to the column heads of df_tst

In [9]:
# The large test data frame that will be used to accept values of each from both 770 N 970 will be built on the basis of the df_ib_tst_nms, but this data bugs be in form of a strings.
# df['colname'].astype(str).tolist       # sign This is the sample given from copilot that is modified below for my situation 

ib_tst_lst = df_ib_tst_nms[0].astype(str).tolist()
# verify 
# print(ib_tst_lst)

In [10]:
# this makes the template for the ib_tst data For both the 770 and the 970 It is made up of a 770 and a 970 with duplicates eliminated in a list of strings

df_ib_tst = pd.DataFrame(
    data = [[""] * len(df_ib_tst_lst)] * 2,
    columns = ib_tst_lst
)
#df_ib_tst_nms
df_ib_tst

NameError: name 'df_ib_tst_lst' is not defined

In [ ]:
# Tests 
df_ib_tst["ID"]


In [ ]:
df_ib_tst.columns


In [ ]:
df_ib_tst["ID"]

# The following is the procedure to edit_ This structure of ” ib_tst”

## data frame of the main data storage_“ib_tst"

###  Make the edits to the structure of the main data spreadsheet that represents structure

### Then the CSV file into the worksheet to display the changes in Jupiter lab

In [ ]:
df_loaded = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/ib_tst.csv")
df_loaded.columns

In [ ]:
folder_path = "/home/bhuns/JL_2/data/ib_tst/ib_tst.csv"
df_ib_tst.to_csv(folder_path, index=False)
df_ib_tst

In [ ]:
df_loaded["Age"]



In [ ]:
#==================================================================================================================== Oh

###


# Align the columns via index suggestions from Copilot to insert new data into the major data frame

Put the creation of a data frame into its own worksheet

# ==============================================================================================================================================

## Calculate the *df_ib_tst* column names from the beta column names added to the 770 column names to the 970 add column names

## Create *df_ib_fls_tmplt* using the *df_ib_fls* column names.

## Then save it to pickle as df_ib_fls_tmplt

## Then save it to df_ib_fls.csv  to start a new data set

# Import the ib770 & ib970 to **df_ib_fls** These will be fill in column by column using the column name as a reference and added row by row. 

## Import *df_ib_fls.csv*

##  Import *ib97.csv and remove numbers from* And apend to *df_ib_fls* if timestamp does not exist 

##  Import *ib77.csv*  And apend to *df_ib_fls* if timestamp does not exist 

## 

## 

In [ ]:
ib_fls_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib_fls",
    pattern="*"
)
df_ib_fls_raw = pd.DataFrame(ib_fls_raw)

# verify
# print( df_ib_fls_raw["2. ID"])


In [ ]:
# verify
# print( df_ib_fls_raw["2. ID"])

# df_col_nm_lst =["",""]
df_col_nm_lst = pd.DataFrame( df_ib_fls_raw.columns)
# verify
print(df_col_nm_lst)

# Remove **numbers** from col_nms   ok

In [ ]:
# STRIPS COL NMBRS  column #s from col heads

def strip_col_numbers(col_names):
    """
    Remove leading numbers and dots from column names.
    Example: '244. 50kHz-Whole Body Phase Angle_Z score' 
             -> '50kHz-Whole Body Phase Angle_Z score'
    """
    return [re.sub(r'^\d+\.\s*', '', name) for name in col_names]

# Run this for both the 970 and the 770 data imports
# col_names = C.columns
# df_ib_fls_raw_nm = pd.DataFrame(strip_col_numbers(col_names))
# verify
# print(df_ib_fls_raw_nm)

# Remove Duplicates so each col in 770 and 970 Have one unique Column In this overall *df_ib* dataframe   OK

In [ ]:
df_ib_fls_raw_nm = df_ib_fls_raw_nm.drop_duplicates(keep="first").reset_index(drop=True)


In [ ]:
# verify
# 
print(df_ib_fls_raw_nm)

In [ ]:
print(df_ib_fls_raw)